### Merkle Tree


In [226]:
from hashlib import sha256

class MerkleTree:
    def __init__(self, leaves: list[int]):
        assert len(leaves) & (len(leaves) - 1) == 0, "Leaves must be a power of 2"
        self.leaves = leaves
        self.tree = []
        self.root = self._build_tree()

    def get_path(self, leaf_index: int) -> list[int]:
        path = []
        layer_index = 0
        path.append(self.tree[layer_index][leaf_index])
        while layer_index < len(self.tree) - 1:
            path.append(self.tree[layer_index][leaf_index + 1 if leaf_index % 2 == 0 else leaf_index - 1])
            layer_index += 1
            leaf_index = leaf_index // 2
        return path


    def _build_tree(self) -> list[int]:
        curr_layer = [sha256(str(leaf).encode()).hexdigest() for leaf in self.leaves]
        self.tree.append(curr_layer)
        while len(curr_layer) > 1:
            next_layer = self._build_layer(curr_layer)
            self.tree.append(next_layer)
            curr_layer = next_layer
        return curr_layer[0]
        

    def _build_layer(self, layer: list[int]) -> list[int]:
        new_layer = []
        for i in range(0, len(layer), 2):
            new_layer.append(sha256((layer[i]+ layer[i + 1]).encode()).hexdigest())
        return new_layer


In [227]:
# Test

from pprint import pprint

m = MerkleTree([1,2,3,4,5,6,7,8])
pprint([[b[:4] for b in a] for a in m.tree])
print(m.root)
pprint([a[:4] for a  in m.get_path(0)])

[['6b86', 'd473', '4e07', '4b22', 'ef2d', 'e7f6', '7902', '2c62'],
 ['33b6', '1365', '4358', 'ada1'],
 ['85df', 'e0e2'],
 ['c274']]
c27450cd3fd4df029145f3437ae9c381e0ae55e8400de06cb973005b36d7b222
['6b86', 'd473', '1365', 'e0e2']


### Hash Chain

In [228]:
import random

class HashChain:
    def __init__(self, seed: str):
        self.chain = [seed]
        self.random_generator = random.Random(seed)

    def get_next_random(self):
        nonce = self.random_generator.randbytes(4)
        self.chain.append(sha256(self.chain[-1].encode() + nonce).hexdigest())
        return int(self.chain[-1], 16)

    def get_all_hashes(self):
        return self.chain

### Channel

In [229]:
from queue import Queue

class Channel:
    def __init__(self, F):
        """Initialize the channel with an optional random seed."""
        self.messages = Queue()
        self.F = F

    def send(self, message: dict[str, int | str]):
        """Send (store) a message in the channel."""
        self.messages.put(message)

    def receive(self):
        """Receive (retrieve) the last message sent."""
        return self.messages.get()
    
    def get_all_messages(self):
        """Retrieve all messages exchanged."""
        return list(self.messages.queue)
    
    def _get_hash(self):
        return sha256("".join(map(lambda x: x["data"], self.messages.queue)).encode()).hexdigest()
    
    def receive_random_field_element(self, title: str = "Random Field Element"):
        # TODO change with nonce
        """Generate a random f"""
        random_field_element = self.F(int(self._get_hash(), 16))
        self.messages.put({
            "title": title,
            "data": str(random_field_element)
        })
        return random_field_element

### Fibonacci function

In [230]:
def fibonacci(a=1, size=1001) -> list[int]:
    fib_list = [1, a]
    for _ in range(2, size):
        fib_list.append(fib_list[-1] + fib_list[-2])
    return fib_list 

In [231]:
print(fibonacci())

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 63245986, 102334155, 165580141, 267914296, 433494437, 701408733, 1134903170, 1836311903, 2971215073, 4807526976, 7778742049, 12586269025, 20365011074, 32951280099, 53316291173, 86267571272, 139583862445, 225851433717, 365435296162, 591286729879, 956722026041, 1548008755920, 2504730781961, 4052739537881, 6557470319842, 10610209857723, 17167680177565, 27777890035288, 44945570212853, 72723460248141, 117669030460994, 190392490709135, 308061521170129, 498454011879264, 806515533049393, 1304969544928657, 2111485077978050, 3416454622906707, 5527939700884757, 8944394323791464, 14472334024676221, 23416728348467685, 37889062373143906, 61305790721611591, 99194853094755497, 160500643816367088, 259695496911122585, 420196140727489673, 679891637638612258, 110008777836

### Polynomial f(x)

In [232]:
P: int =3221225473
F = GF(P)
N = 8192

g = F(1734477367)
gamma = g ** 8
gamma_group = [gamma ** i for i in range(1001)]

fibonacci_seq = fibonacci()

R.<x> = PolynomialRing(F)

f = R.lagrange_polynomial(zip(gamma_group, fibonacci_seq))

### Bit Reverse Order

In [233]:
def bit_reverse(n, width):
    rev = 0  # This will store the reversed bits
    for i in range(width):
        rev = (rev << 1) | (n & 1)  # Shift left and take LSB of n
        n >>= 1  # Shift n to the right
    return rev

def bit_reverse_permutation(N):
    """Generate bit-reversed order for N elements."""
    width = int(math.log(N, 2))
    return [bit_reverse(i, width) for i in range(N)]

bit_reverse_order = bit_reverse_permutation(N)

### w < g >

In [234]:
w = F.multiplicative_generator()

g_group = [g ** i for i in range(N)]

wg = [w * gi for gi in g_group]
wg_merkle_order = [wg[i] for i in bit_reverse_order]

### Commit f(x) on LDE

In [235]:
channel = Channel(F)

f_wg = [f(x) for x in wg_merkle_order]

merkle_f_wg = MerkleTree(f_wg)

channel.send({'title': 'f(x) on LDE', 'data': merkle_f_wg.root})

### Polynomial h(x)

In [236]:
d_x = 1
for gamma_i in gamma_group[:-2]:
    d_x = d_x * (x - gamma_i)

F_x = f(x) + f(gamma * x) - f((gamma**2) * x)

h_x = F_x // d_x

In [237]:
h_wg = [F_x(x) / d_x(x) for x in wg_merkle_order]

# Sanity check 
print([h_x(x) for x in wg_merkle_order][:10])
print(h_wg[:10])

[1860826706, 1845649622, 290025805, 195225050, 2156032599, 1550443729, 1756057239, 1950419089, 356807959, 128442896]
[1860826706, 1845649622, 290025805, 195225050, 2156032599, 1550443729, 1756057239, 1950419089, 356807959, 128442896]


### Boundary Constraints

In [238]:
Y = fibonacci_seq[-1]

t0_x = (f(x) - 1) / (x - gamma ** 0)
t1_x = (f(x) - Y) / (x - gamma ** 1000)

In [239]:
t0_wg = [(f(i) - 1) / (i - gamma ** 0) for i in wg_merkle_order]
t1_wg = [(f(i) - Y) / (i - gamma ** 1000) for i in wg_merkle_order]

# Sanity check
print([t0_x(x) for x in wg_merkle_order][:10])
print(t0_wg[:10])
print("")
print([t1_x(x) for x in wg_merkle_order][:10])
print(t1_wg[:10])

[2384530853, 599226013, 2071111314, 1402510381, 2026908630, 951059204, 1906847992, 1092764146, 3083267015, 51241944]
[2384530853, 599226013, 2071111314, 1402510381, 2026908630, 951059204, 1906847992, 1092764146, 3083267015, 51241944]

[2395950328, 2696799910, 124207998, 3010185695, 1771367485, 510712279, 521057469, 2771188778, 1200373265, 956436872]
[2395950328, 2696799910, 124207998, 3010185695, 1771367485, 510712279, 521057469, 2771188778, 1200373265, 956436872]


### Composition Polynomial

In [240]:
beta_0 = channel.receive_random_field_element()
beta_1 = channel.receive_random_field_element()
beta_2 = channel.receive_random_field_element()

cp0_x = beta_0 * h_x + beta_1 * t0_x + beta_2 * t1_x

In [241]:
cp0_wg = [ beta_0 * h_wg[i] + beta_1 * t0_wg[i] + beta_2 * t1_wg[i] for i in range(len(wg))]

# Sanity check
print([cp0_x(x) for x in wg_merkle_order][:10])
print(cp0_wg[:10])

[2707245237, 2801024926, 1222765448, 588653826, 2002937464, 1642553073, 2268095886, 2153631073, 1601448270, 1303100525]
[2707245237, 2801024926, 1222765448, 588653826, 2002937464, 1642553073, 2268095886, 2153631073, 1601448270, 1303100525]


### FRI

In [242]:
fri_layers = []
degree = R(cp0_x).degree()
if degree % 2 != 0: 
    degree += 1

curr_cp = [cp0_x(x) for x in wg_merkle_order]
while degree > 0:
    fri_layers.append(curr_cp)
    # Commit curr layer
    merkle_curr_cp = MerkleTree(curr_cp)
    channel.send({
        "title": f"commit for layer {len(fri_layers) - 1} of FRI",
        "data": merkle_curr_cp.root,
    })
    random_x = channel.receive_random_field_element()
    curr_cp = [((curr_cp[i] + curr_cp[i + 1]) / 2) + \
                     random_x * (curr_cp[i] - curr_cp[i + 1]) / (2 * wg_merkle_order[i]) for i in range(len(curr_cp))[::2]]
    degree = degree // 2

constant = curr_cp[0]
channel.send({"title:": "constant of last FRI layer", 
              "data": str(constant),
              })

In [243]:
print(fri_layers)
print("Merkle Root")
print(curr_cp[0])

[[2707245237, 2801024926, 1222765448, 588653826, 2002937464, 1642553073, 2268095886, 2153631073, 1601448270, 1303100525, 2600562454, 1172988919, 1853806351, 2937920145, 51353313, 3154977765, 172448295, 2039740447, 927969672, 1905784717, 2794531855, 665667924, 3219511666, 1383228476, 756587836, 1487783672, 2776320175, 1503089550, 1036207733, 1481817349, 2114100284, 2017850723, 546043133, 1963875706, 2209486806, 2900178218, 2491368981, 600120228, 3053957442, 2190346381, 2716081732, 2194888639, 1514389343, 2311129992, 715564386, 2298434942, 2725633831, 2340457695, 3181639041, 2725335377, 2552967056, 2491802633, 346896521, 970363316, 1530336522, 1321269907, 1784322818, 281967899, 2509898359, 369991127, 1747554155, 3057732317, 2819133441, 301176190, 535640216, 1911863201, 615188001, 53509647, 761889524, 1425702436, 744834808, 1234154143, 2953447789, 1143254156, 71235629, 2807778463, 775029325, 2383974640, 2361600289, 664529980, 2886122602, 626817607, 2946489991, 2003382451, 1017622968, 3778

In [244]:
# Sanity check
print(curr_cp)

[2991301305, 2991301305, 2991301305, 2991301305, 2991301305, 2991301305, 2991301305, 2991301305]


In [245]:
pprint(channel.get_all_messages())

[{'data': 'd524fba68e1263f084afe501c715dca773c616d5a8c52303322985d32c94ea56',
  'title': 'f(x) on LDE'},
 {'data': '592641122', 'title': 'Random Field Element'},
 {'data': '174315803', 'title': 'Random Field Element'},
 {'data': '2915487417', 'title': 'Random Field Element'},
 {'data': '7f4339f4103915d44198554abd703d5a697c16925b72f4fe5c7523c9db60fefb',
  'title': 'commit for layer 0 of FRI'},
 {'data': '2799795021', 'title': 'Random Field Element'},
 {'data': '0c13eae310a42f89940d76075cad7a5dd9f5f65072d5c3aa03c36839e62189cd',
  'title': 'commit for layer 1 of FRI'},
 {'data': '756169561', 'title': 'Random Field Element'},
 {'data': 'b91ab11483d1e3003bae2fa41c94dbfcb7cdd67fee944d49e5f0df1c4f900dc8',
  'title': 'commit for layer 2 of FRI'},
 {'data': '1634783973', 'title': 'Random Field Element'},
 {'data': 'b5be20a766e4a2fcd7ee58a4450f6bacd2f08f9bb873f6006da0014a79b29159',
  'title': 'commit for layer 3 of FRI'},
 {'data': '3001619057', 'title': 'Random Field Element'},
 {'data': '04489

### Decommit Phase

# TIOTA


In [246]:




cp_prev = R(cp_0_x)
cps = []
print("channel before FRI", channel.get_all_messages())
while cp_prF_x_on_wg[i]ev.degree() > 0:
    cp_prev_on_wg = [cp_prev(i) for i in wg]

    merk = MerkleTree([cp_prev_on_wg[i] for i in bit_reverse_order])
    commit = merk.root
    channel.send({'title': f'commit for degree {cp_prev.degree()}', 'data': commit})
    
    random_x = channel.receive_random_field_element()
    cp_next = (cp_prev(x) + cp_prev(-x)) // 2 + random_x * (cp_prev(x) - cp_prev(-x)) // (2 * x)
    cp_next = sum(coef * x^(exp // 2) for exp, coef in R(cp_next).dict().items())
    cps.append(cp_prev)
    cp_prev = cp_next

channel.send({'title': 'final_commit', 'data': cp_prev})

def verify_merkle_path(merkle_root: str, merkle_path: list[str]):
    root = merkle_path[0]
    for sibling in merkle_path[1:]:
        root = sha256((root + sibling).encode()).hexdigest()
    return root == merkle_root




SyntaxError: invalid syntax (2599989602.py, line 4)

## Decommit Phase

In [ ]:
from random import randint, seed
seed(42069)

rand_idx = randint(0, len(wg))
rand_x = wg[rand_idx]
f_x_at_idx = f(rand_x)
f_x_at_g_idx = f(gamma * rand_x)
f_x_at_g2_idx = f(gamma**2 * rand_x)

channel.send({'title': 'decommit phase 0 ', 'data': {'result': f_x_at_idx, 'path': merkle_f_wg.get_path(rand_idx)}})

for i in range(len(cps)):
    cp = cps[i]
    cp_at_idx = cp(rand_x)
    cp_at_neg_idx = cp(-rand_idx)




8192